In [19]:
import RNA
import numpy as np

import subprocess
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import SVG, display
from collections import Counter
from collections import defaultdict

import difflib
import sys
import os
import random
import string
import time

# from helper import print_moves

import pandas as pd

sys.path.append('../')
from pretty_print_path import print_moves
import findpath_librna
import findpath


In [86]:
# 100 nt with 2 inner sections
# sequence = "CGCAUCUCUUUAGGGUAUGAAAUGUUAUAUGCUACGGGAACAAUGCCGACCUUCGGAGACCUAAGGAAUACGUCUUUCGAGCGGAAGGAUUCCUCGUUCA"
# s1 = ".(((((((..(((.((((((....)))))).))).))))....)))(((...)))..(((...((((....(((((((.....))))))))))).))).."
# s2 = "....((.((.(((.((((((....)))))).))).))))......((((...)))).(((...(((((....((((((.....))))))))))).))).."

# sequence = "AAAAUAAUGUACCGGACAUUCGCGCACGACCACCAUAUGGCAGAGCAUGUGUCUGUGGACCCACUAUAGCUGGGGCGCUUAACCCCAGAAAAGUAUCUUCGGUCUAUGCCUCACACGCAGCCUCCUAUUAGCAGCUCUCCUGGCCCACAAUUUUAUUAAAAGUCCAAGUUGGACUGACAAAACGCGUGCGGUGUCCUAGGGAUUGGUGGCAUAACCAGCGGUUUAAAAGCUGUGUAUAUCCGCAGCAAAUCACCGGAAAGCGGCGUUAUUAGCACCACAAAUUGAUGGUUGGUACGAGUACAAUUGCGCCGCAUAAAACCAGAGAUUCUACCCUCAAUCGGUUCUUAAGACGUACUGCGCGUUUCACCAGACCACAAUGCAGGGCGGCACCGUUAGGCAACACAACGAGACUACUCAUGCACAUAAGGAAGGUUAUCGCCAUAGACAUGGCGCGGCAGCGCAGAAUGUUUAAAUCUAAAUCUGGUAUGGGAGGCGUGCCCGUUGGUAUGAAGAAAUUUGCUGGGAGAAAAAGUCUAAGGCCUUGAAUCCGGCGGGUCUUAAUACUUACCUACAAAAUCAUCAGGCUGUACUUCCUGUAUC"
# s1 =       "........(((((((((((((((((..((((.((.((((((((((...((.((((((((((....(((.((((((.......)))))).....)))....))))..(((((...(((..(((((((....(((......((((.................((((((...))))))........((((((((((((.(((((((((((...........((((......((((((......)))))).....))))....(((((((((((.(.((((((......))..)))).).)))).....))))))).........(((........))).))))))))))).)))..)))))))))....)))).......))))))).)))..))).))))).))))..)).))..))).))).)))).))..))))..((((((....)))))).....)))).)))))............))))))))((((((((.(((.(.((((.........((((..(((((.....((.((((((((((....))..)))))))))).))).))..)))))))).).))))))..)))))....."
# s2 =       ".............((((((((((((.(((((.((.((((((((((...((.((((((((((...(((..((((((.......))))))....))).....))))..(((((...(((..(((((((....(((......((((.................((((((...))))))........((((((((((((.(((((((((((((.....))..((((......((((((......)))))).....))))....((((((((((((..((((........)))))))))...........))))))).........(((........))).))))))))))).)))..)))))))))....)))).......))))))).)))..))).))))).))))..)).))..))).))).)))).))..)))...((((((....))))))))...)))).))))))))............(((((((((((((.(((.((.(((((.((((..(((((((((.......(((...)))......))))))))))))).))))).))..............)))))..)))))))))))"


# no sections 300
sequence = "UCACGACACCCCUCAACUAUAACAACGGUCCGUACAUACUAGCCCUGCAAUGGAACGGGCAGGGCCAGCACAGGUGGGGCGCCCGCUUGGGGGAUCAAAUGUGUGAUCCAGAUACUUUAGACGCGUGCAGAACUUUUUAGAUCGAUCAGUGGGAACAGGCAUUGAUUAUGAAAUCAAUUAGGGGGUUUAGGACCGCACCACAAACUGCGGGAGGGCACGCUUUGGUUCCUGUGUUACGCUAAUCCUCUAGCCACGGAGGGCUUCUUCGUACAAUGAUUGGGUUACCAGGGUUCCAGUGUG"
s1 = ".........(((((.(((........)))............(((((((..........))))))).....((((((((...))))))))..((((((......))))))......(((((((.(.(........(((((....((((((((........)))))))).))))).......).).))))))).(((((........))))))))))((((((..((.(((((.((.((.((((((........((((((.....)))))).....)))))))).)))))))..))))))))"
s2 = ".(((...((((........((((..(((((.((((...((.(((((((..........))))))).))..((((((((...))))))))((((((.....(((((((.(((..((..((((.((((((.(((((((((((((.((((((((........)))))))).....)))...))))))))))....(((((........)))))....))))))))))))..))).)))))))..)))))).((((......)))).....))))...))))).))))...))))....))).."

# 3 ext loops
# sequence = "AACCCGCGAUAAGUUGGCGCUUGUCCCACUCCGUAAACCUGUGUCUCUCAGGCGGUUACCCGAUAGAAGGCAGUAGGAUGUAUCACCCCCCACCGACUCCACUAUACAACGAAACCGGCCCUUUGUGAAUAAACGCUCUCAGUUAGAUUGAGCGGCGCAACAGAACCAGAUCGAUUCCCGGGGCAGUAGUGUCCUACUGCCCAACCCGGUAGAGGGAUCGAGUGGUCUAGGCGAAUGAUACGAGAGGCUCACUGGGGACGAUGGGAGUGAUCUACUCGAUGUUGCGUCCGAUCACAUCCA"
# s1 =       "....(((.(.....).))).(((((((............((.(((((((.(((((((.....((((..((.(((.((.((..........)))).))))).)))).......))))).)).............(((((...........))))).(((...(((.(((..(((((((((((((((((((....))))))))....))).....)))))))).))))))..)))........))))))).))...)))))))...(((((...)))))(((((...........))))).."
# s2 =       "....(((.(.....).))).(((((((............((.(((((((.(((((((...((((((..((.(((.((.((..........)))).))))).))))....)).)))).))).((((....))))(((((...........))))).(((...((.((((..(((((((((((((((((((....)))))))))....)).....)))))))).))))))..)))........))))))).))...)))))))((((.((((((....((......))...)))))).))))"


# sequence = "AUAAUGUACCGGACAUUCG"
# s1       = "..(((((.....))))).."
# #             X             X
# # s2       = "...((((.....))))..."   # delete
# s2       = "..((((((...)))))).."   # insert

search_width_multiplier = 20
search_width_multiplier = 4


In [87]:
# call with search width multiplier:
fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=search_width_multiplier, mp=True)

# call with fixed search width integer:
# fp = findpath.findpath_single(sequence, s1, s2, search_width=50, mp=mp)

# call with added model detail dictionary:
# fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=search_width_multiplier, 
#     mp=mp, model_details={"temperature": -14.5,})


result = fp.get_en()/100.0
path = fp.get_path()



In [83]:
# print_moves(sequence, s1, s2, path, convert_to_float=True)

In [88]:
def find_moves(s_ptable, t_ptable):
    """
    generator function, yields possible structures 1 move away
    from the original structure by finding fitting i and j with
    RNA pair and loop tables
    s_ptable: current ptable
    t_ptable: s2 end ptable
    """
    # loop table
    ls = RNA.loopidx_from_ptable(s_ptable)

    for i in range(len(s_ptable)):
        if i == 0:
            continue

        if s_ptable[i] == 0 and t_ptable[i] > i:
            j = t_ptable[i]
            # found j has to be empty and currently on the same loop as i
            if s_ptable[j] == 0 and ls[i] == ls[j]:
                yield i, j
        # test for bp removal: i has to be paired with a different j in s2
        j = s_ptable[i]
        # dont remove things which are present in s2
        if s_ptable[i] > i and s_ptable[i] != s_ptable[j] and\
                s_ptable[i] != t_ptable[i] and s_ptable[j] != t_ptable[j]:
            yield -i, -j



s = s1
pt2 = list(RNA.ptable(s2))
fc = RNA.fold_compound(sequence)

for e, (a,b, en) in enumerate(path):
    

    if (a,b) == (0,0):
        continue

    


    # check where we can go, compare with our best move. 
    pt = list(RNA.ptable(s))


    # check available moves, save them, sort them
    
    avail_moves = []

    found_pos = None

    for pos, (i,j) in enumerate(find_moves(pt, pt2)):    
        next_en = fc.eval_move_pt(pt, i, j)
        # mark where we found our move

        found = (i,j) == (a,b)
        avail_moves.append((i, j, next_en, found))


    avail_moves.sort(key=lambda x: x[2])
    found_list = [x[3] for x in avail_moves]

    # find where our move is after sorting
    found_pos = found_list.index(True)
    rel_pos = found_pos * 1.0 / len(found_list)

    print (e, a,b, 'found at pos:', found_pos, 'of', len(avail_moves), ':',  1-rel_pos)
    # print (avail_moves, a, b)

    
    # if e==63:
    #     print (avail_moves)

    # update s for the next iteration
    
    if a > 0:
        s = s[:a-1] + "(" + s[a:b-1] + ")" + s[b:]
    if a < 0:
        s = s[:-a-1] + "." + s[-a:-b-1] + "." + s[-b:]
    # if not en:
    #     en = round(fc.eval_structure(s), 2)



    # print (s)

    # if e>1:
    #     break

    # break





1 39 68 found at pos: 1 of 59 : 0.9830508474576272
2 40 67 found at pos: 0 of 58 : 1.0
3 -224 -294 found at pos: 15 of 57 : 0.736842105263158
4 -225 -293 found at pos: 0 of 56 : 1.0
5 -116 -191 found at pos: 0 of 55 : 1.0
6 -139 -169 found at pos: 3 of 54 : 0.9444444444444444
7 -97 -104 found at pos: 1 of 53 : 0.9811320754716981
8 -117 -190 found at pos: 0 of 52 : 1.0
9 -138 -170 found at pos: 5 of 51 : 0.9019607843137255
10 -126 -181 found at pos: 0 of 50 : 1.0
11 -137 -171 found at pos: 0 of 49 : 1.0
12 -136 -172 found at pos: 0 of 48 : 1.0
13 -135 -173 found at pos: 0 of 47 : 1.0
14 -18 -27 found at pos: 9 of 53 : 0.8301886792452831
15 -16 -29 found at pos: 0 of 52 : 1.0
16 -17 -28 found at pos: 0 of 51 : 1.0
17 -118 -189 found at pos: 0 of 50 : 1.0
18 -119 -188 found at pos: 0 of 49 : 1.0
19 -120 -187 found at pos: 0 of 48 : 1.0
20 -121 -186 found at pos: 0 of 47 : 1.0
21 -122 -185 found at pos: 5 of 46 : 0.8913043478260869
22 -124 -183 found at pos: 0 of 45 : 1.0
23 140 175 found 